In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = set(stopwords.words("english")) | set(stopwords.words("russian"))
from sklearn.metrics import mean_squared_error, accuracy_score
import re
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
    100% |████████████████████████████████| 51kB 2.5MB/s 
    100% |████████████████████████████████| 7.1MB 4.4MB/s 


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving train.csv to train.csv


In [0]:
train = pd.read_csv('train.csv')

**Basic Feature Extraction **

  Number of Words



In [0]:
train['word_count'] = train['text'].apply(lambda x: len(str(x).split(" ")))
train[['text','word_count']].head()

Number of characters

In [0]:
train['char_count'] = train['text'].str.len() ## this also includes spaces
train[['text','char_count']].head()

Average Word Length

In [0]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

train['avg_word'] = train['text'].apply(lambda x: avg_word(x))
train[['text','avg_word']].head()

Number of stopwords

In [0]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

train['stopwords'] = train['text'].apply(lambda x: len([x for x in x.split() if x in stop]))
train[['text','stopwords']].head()

Number of numerics

In [0]:
train['numerics'] = train['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
train[['text','numerics']].head()

Number of Uppercase words

In [0]:
train['upper'] = train['tweet'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
train[['tweet','upper']].head()

Number of special characters

In [0]:
train['exclamation'] = train['text'].apply(lambda x: len([x for x in x.split() if x.startswith('!')]))
train[['text','hastags']].head()

In [0]:
df = pd.read_csv('train.csv')
def clean(text):
    
    text = re.sub("[^а-яА-Яa-zA-Z]"," ", text)
    words = text.lower().split()
    words = [w for w in words if not w in stops]
    words = [morph.parse(w)[0].normal_form for w in words ]
    return(words)
  
for i in tqdm(range(len(df))):
    df.iloc[i]['text'] = clean(df.iloc[i]['text'])

df.sample(frac = 1, random_state = 42).head()
df.to_csv('clean.csv', sep=';', encoding='utf-8')

100%|██████████| 3485/3485 [27:41<00:00,  1.20it/s]


In [0]:

train_df = pd.read_csv("clean.csv", sep=";")

In [0]:
y = [int(string[9:]) for string in df.author.values] # remove "__label__" from string and convert to int

xtrain, xvalid, ytrain, yvalid = train_test_split(train_df.text.values, y, 
                                                  random_state=42, 
                                                  test_size=0.1,)

In [0]:
tfv = TfidfVectorizer(min_df=3, analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1) 

tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)

In [0]:
regr_1 = LogisticRegression(C=1000, penalty = 'l2' )
regr_1.fit(xtrain_tfv, ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


NameError: ignored

In [0]:
predictions_1 = regr_1.predict(xvalid_tfv)
accuracy_score(predictions_1, yvalid)

0.9054441260744985

Пробуем word2vec


In [0]:
from gensim.models import word2vec
model = word2vec.Word2Vec(df['text'], size=300, window=10, workers=4)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
class mean_vectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(w2v.values())))

    def fit(self, X):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

In [0]:
xtrain_mean = mean_vectorizer(w2v).fit(xtrain).transform(xtrain)
xvalid_mean = mean_vectorizer(w2v).fit(xvalid).transform(xvalid)
xtrain_mean.shape

(3136, 300)

In [0]:
regr_2 = LogisticRegression()
regr_2.fit(xtrain_mean, ytrain)
predictions_2 = regr_2.predict(xvalid_mean)
accuracy_score(predictions_2, yvalid)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.11174785100286533

In [0]:
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_mean, ytrain)
predictions_3 = clf.predict(xvalid_mean)



NameError: ignored

In [0]:
print ("accuracy: %0.3f " % accuracy_score(yvalid, predictions_3))

accuracy: 0.246 


In [0]:
train_df['text']